In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import io

## Основные функции расчета

In [2]:
class SphereRenderer:
    def __init__(self, W, H, Wres, Hres, sphere_center, sphere_radius, observer_pos):
        """
        Инициализация рендерера сферы
        
        Параметры:
        W, H - ширина и высота экрана в мм
        Wres, Hres - разрешение в пикселях
        sphere_center - координаты центра сферы (x, y, z)
        sphere_radius - радиус сферы в мм
        observer_pos - позиция наблюдателя (x, y, z)
        """
        self.W = W
        self.H = H
        self.Wres = Wres
        self.Hres = Hres
        self.sphere_center = np.array(sphere_center, dtype=float)
        self.sphere_radius = sphere_radius
        self.observer_pos = np.array(observer_pos, dtype=float)
        
        # Пиксели должны быть квадратными
        self.pixel_size_w = W / Wres
        self.pixel_size_h = H / Hres
        
        self.brightness_data = None
        self.max_brightness = 0
        self.min_brightness = float('inf')
        
    def ray_sphere_intersection(self, ray_origin, ray_direction):
        """
        Находит пересечение луча со сферой
        Возвращает точку пересечения или None
        """
        oc = ray_origin - self.sphere_center
        a = np.dot(ray_direction, ray_direction)
        b = 2.0 * np.dot(oc, ray_direction)
        c = np.dot(oc, oc) - self.sphere_radius ** 2
        discriminant = b * b - 4 * a * c
        
        if discriminant < 0:
            return None
        
        t = (-b - np.sqrt(discriminant)) / (2.0 * a)
        if t < 0:
            t = (-b + np.sqrt(discriminant)) / (2.0 * a)
            if t < 0:
                return None
        
        return ray_origin + t * ray_direction
    
    def blinn_phong(self, point, normal, view_dir, light_sources, ka, kd, ks, shininess):
        """
        Модель освещения Блинн-Фонга
        
        Параметры:
        point - точка на поверхности
        normal - нормаль в точке
        view_dir - направление к наблюдателю
        light_sources - список источников света [(pos, intensity), ...]
        ka - коэффициент фонового освещения
        kd - коэффициент диффузного отражения
        ks - коэффициент зеркального отражения
        shininess - показатель блеска
        """
        ambient = ka
        diffuse = 0.0
        specular = 0.0
        
        for light_pos, intensity in light_sources:
            # Направление к источнику света
            light_dir = light_pos - point
            distance = np.linalg.norm(light_dir)
            light_dir = light_dir / distance
            
            # Ламбертовская диаграмма излучения
            # Интенсивность убывает с квадратом расстояния
            light_intensity = intensity / (distance ** 2)
            
            # Диффузная составляющая (закон Ламберта)
            n_dot_l = max(np.dot(normal, light_dir), 0.0)
            diffuse += kd * light_intensity * n_dot_l
            
            # Зеркальная составляющая (Блинн-Фонг)
            if n_dot_l > 0:
                half_dir = (light_dir + view_dir) / np.linalg.norm(light_dir + view_dir)
                n_dot_h = max(np.dot(normal, half_dir), 0.0)
                specular += ks * light_intensity * (n_dot_h ** shininess)
        
        return ambient + diffuse + specular
    
    def render(self, light_sources, ka=0.1, kd=0.6, ks=0.3, shininess=32):
        """
        Рендеринг сферы с заданными источниками света
        
        light_sources - список источников света [(pos, intensity), ...]
        """
        self.brightness_data = np.zeros((self.Hres, self.Wres))
        self.max_brightness = 0
        self.min_brightness = float('inf')
        
        # Трехмерный массив для хранения информации о точках
        self.point_data = np.full((self.Hres, self.Wres, 3), np.nan)
        
        for i in range(self.Hres):
            for j in range(self.Wres):
                # Координаты пикселя на экране
                x = -self.W / 2 + (j + 0.5) * self.pixel_size_w
                y = self.H / 2 - (i + 0.5) * self.pixel_size_h
                z = 0  # Экран в плоскости z=0
                
                screen_point = np.array([x, y, z])
                
                # Направление луча от наблюдателя к пикселю
                ray_dir = screen_point - self.observer_pos
                ray_dir = ray_dir / np.linalg.norm(ray_dir)
                
                # Пересечение со сферой
                intersection = self.ray_sphere_intersection(self.observer_pos, ray_dir)
                
                if intersection is not None:
                    # Нормаль в точке пересечения
                    normal = (intersection - self.sphere_center) / self.sphere_radius
                    
                    # Направление к наблюдателю
                    view_dir = self.observer_pos - intersection
                    view_dir = view_dir / np.linalg.norm(view_dir)
                    
                    # Расчет яркости по модели Блинн-Фонга
                    brightness = self.blinn_phong(
                        intersection, normal, view_dir, light_sources,
                        ka, kd, ks, shininess
                    )
                    
                    self.brightness_data[i, j] = brightness
                    self.point_data[i, j] = intersection
                    
                    if brightness > self.max_brightness:
                        self.max_brightness = brightness
                    if brightness < self.min_brightness:
                        self.min_brightness = brightness
        
        return self.brightness_data
    
    def get_brightness_at_points(self, num_points=3):
        """
        Возвращает значения яркости в случайных точках сферы
        """
        if self.brightness_data is None:
            return []
        
        results = []
        mask = ~np.isnan(self.point_data[:, :, 0])
        valid_indices = np.argwhere(mask)
        
        if len(valid_indices) < num_points:
            num_points = len(valid_indices)
        
        selected_indices = valid_indices[np.random.choice(len(valid_indices), num_points, replace=False)]
        
        for idx in selected_indices:
            i, j = idx
            point = self.point_data[i, j]
            brightness = self.brightness_data[i, j]
            results.append((point, brightness))
        
        return results
    
    def normalize_and_convert(self):
        """
        Нормализует яркость к диапазону 0-255
        """
        if self.max_brightness > 0:
            normalized = (self.brightness_data / self.max_brightness * 255).astype(np.uint8)
        else:
            normalized = np.zeros((self.Hres, self.Wres), dtype=np.uint8)
        
        return normalized
    
    def save_image(self, filename):
        """
        Сохраняет изображение в файл
        """
        normalized = self.normalize_and_convert()
        img = Image.fromarray(normalized, mode='L')
        img.save(filename)
        print(f"Изображение сохранено в {filename}")

## Интерактивный интерфейс с виджетами

In [ ]:
# Создание виджетов для параметров
output = widgets.Output()

# Параметры экрана
w_slider = widgets.IntSlider(value=5000, min=100, max=10000, step=100, description='Ширина (мм):')
h_slider = widgets.IntSlider(value=5000, min=100, max=10000, step=100, description='Высота (мм):')
wres_slider = widgets.IntSlider(value=400, min=200, max=800, step=50, description='Wres (пикс):')
hres_slider = widgets.IntSlider(value=400, min=200, max=800, step=50, description='Hres (пикс):')

# Параметры сферы
sphere_x = widgets.FloatSlider(value=0, min=-10000, max=10000, step=100, description='Сфера X:')
sphere_y = widgets.FloatSlider(value=0, min=-10000, max=10000, step=100, description='Сфера Y:')
sphere_z = widgets.FloatSlider(value=3000, min=100, max=10000, step=100, description='Сфера Z:')
sphere_r = widgets.FloatSlider(value=800, min=50, max=2000, step=50, description='Радиус:')

# Параметры наблюдателя
observer_z = widgets.FloatSlider(value=-5000, min=-10000, max=-100, step=100, description='Наблюдатель Z:')

# Параметры источника света
light1_x = widgets.FloatSlider(value=-1500, min=-10000, max=10000, step=100, description='Свет X:')
light1_y = widgets.FloatSlider(value=2000, min=-10000, max=10000, step=100, description='Свет Y:')
light1_z = widgets.FloatSlider(value=500, min=100, max=10000, step=100, description='Свет Z:')
light1_i = widgets.FloatLogSlider(value=5000000, base=10, min=4, max=8, step=0.1, description='I0 (Вт/ср):')

# Параметры модели Блинн-Фонга
ka_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='Ka (ambient):')
kd_slider = widgets.FloatSlider(value=0.7, min=0.0, max=1.0, step=0.05, description='Kd (diffuse):')
ks_slider = widgets.FloatSlider(value=0.8, min=0.0, max=1.0, step=0.05, description='Ks (specular):')
shininess_slider = widgets.IntSlider(value=128, min=1, max=512, step=1, description='Shininess:')

# Кнопка рендеринга
render_button = widgets.Button(description='Рассчитать и визуализировать', button_style='success')
save_button = widgets.Button(description='Сохранить изображение', button_style='primary')

# Глобальная переменная для рендерера
renderer = None

def on_render_clicked(b):
    global renderer
    with output:
        output.clear_output(wait=True)
        print("Начинаем расчет...")
        
        # Создание рендерера
        renderer = SphereRenderer(
            W=w_slider.value,
            H=h_slider.value,
            Wres=wres_slider.value,
            Hres=hres_slider.value,
            sphere_center=[sphere_x.value, sphere_y.value, sphere_z.value],
            sphere_radius=sphere_r.value,
            observer_pos=[0, 0, observer_z.value]
        )
        
        # Один источник света
        light_sources = [
            (np.array([light1_x.value, light1_y.value, light1_z.value]), light1_i.value)
        ]
        
        # Рендеринг
        renderer.render(
            light_sources=light_sources,
            ka=ka_slider.value,
            kd=kd_slider.value,
            ks=ks_slider.value,
            shininess=shininess_slider.value
        )
        
        print("Расчет завершен!\n")
        
        # Вывод статистики
        print(f"Максимальная яркость: {renderer.max_brightness:.6f} Вт/м²/ср")
        print(f"Минимальная яркость: {renderer.min_brightness:.6f} Вт/м²/ср\n")
        
        # Яркость в трех точках
        print("Значения яркости в трех случайных точках сферы:")
        sample_points = renderer.get_brightness_at_points(3)
        for idx, (point, brightness) in enumerate(sample_points, 1):
            print(f"  Точка {idx}: ({point[0]:.2f}, {point[1]:.2f}, {point[2]:.2f}) мм, "
                  f"Яркость: {brightness:.6f} Вт/м²/ср")
        
        # Визуализация
        normalized = renderer.normalize_and_convert()
        
        # Создание среза яркости по горизонтали (центральная линия)
        center_row = renderer.Hres // 2
        cross_section = renderer.brightness_data[center_row, :]
        x_coords = np.arange(renderer.Wres)
        
        fig = plt.figure(figsize=(16, 10))
        gs = fig.add_gridspec(3, 2, height_ratios=[2, 2, 1], hspace=0.3, wspace=0.3)
        
        # Изображение на черном фоне
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.imshow(normalized, cmap='gray', vmin=0, vmax=255)
        ax1.set_title('Распределение яркости на сфере (черный фон)', fontsize=12, fontweight='bold')
        ax1.axis('off')
        ax1.axhline(y=center_row, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Линия среза')
        ax1.legend(loc='upper right')
        
        # Цветная визуализация с цветовой шкалой
        ax2 = fig.add_subplot(gs[0, 1])
        im = ax2.imshow(normalized, cmap='hot', vmin=0, vmax=255)
        ax2.set_title('Распределение яркости (цветовая карта)', fontsize=12, fontweight='bold')
        ax2.axis('off')
        plt.colorbar(im, ax=ax2, label='Яркость (0-255)', fraction=0.046)
        
        # График среза яркости
        ax3 = fig.add_subplot(gs[1, :])
        ax3.plot(x_coords, cross_section, linewidth=2, color='cyan', label='Срез яркости')
        ax3.fill_between(x_coords, cross_section, alpha=0.3, color='cyan')
        ax3.set_title('Срез распределения яркости (горизонтальная центральная линия)', fontsize=12, fontweight='bold')
        ax3.set_xlabel('Позиция по ширине (пиксели)', fontsize=10)
        ax3.set_ylabel('Яркость (Вт/м²/ср)', fontsize=10)
        ax3.grid(True, alpha=0.3, linestyle='--')
        ax3.legend(loc='upper right')
        ax3.set_facecolor('#f0f0f0')
        
        # Гистограмма
        ax4 = fig.add_subplot(gs[2, :])
        valid_brightness = renderer.brightness_data[renderer.brightness_data > 0]
        if len(valid_brightness) > 0:
            ax4.hist(valid_brightness, bins=50, color='orange', alpha=0.7, edgecolor='black')
            ax4.set_title('Гистограмма яркости', fontsize=12, fontweight='bold')
            ax4.set_xlabel('Яркость (Вт/м²/ср)', fontsize=10)
            ax4.set_ylabel('Количество пикселей', fontsize=10)
            ax4.grid(True, alpha=0.3, axis='y')
            ax4.set_yscale('log')
        
        plt.suptitle('Анализ освещенности сферы с одним источником света (модель Блинн-Фонга)', 
                     fontsize=14, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.show()

def on_save_clicked(b):
    global renderer
    with output:
        if renderer is None:
            print("\nСначала выполните расчет!")
            return
        
        filename = 'sphere_brightness.png'
        renderer.save_image(filename)
        
        # Сохранение данных яркости
        np.savetxt('brightness_data.csv', renderer.brightness_data, delimiter=',', fmt='%.6f')
        print("Данные яркости сохранены в brightness_data.csv")
        
        # Сохранение среза
        center_row = renderer.Hres // 2
        cross_section_data = np.column_stack((
            np.arange(renderer.Wres),
            renderer.brightness_data[center_row, :]
        ))
        np.savetxt('cross_section.csv', cross_section_data, delimiter=',', 
                   fmt='%d,%.6f', header='Pixel,Brightness', comments='')
        print("Данные среза сохранены в cross_section.csv")

render_button.on_click(on_render_clicked)
save_button.on_click(on_save_clicked)

## Панель управления

In [16]:
# Организация виджетов в группы
screen_box = widgets.VBox([
    widgets.HTML("<h3>Параметры экрана</h3>"),
    w_slider, h_slider, wres_slider, hres_slider
])

sphere_box = widgets.VBox([
    widgets.HTML("<h3>Параметры сферы</h3>"),
    sphere_x, sphere_y, sphere_z, sphere_r
])

observer_box = widgets.VBox([
    widgets.HTML("<h3>Наблюдатель</h3>"),
    observer_z
])

light_box = widgets.VBox([
    widgets.HTML("<h3>Источник света</h3>"),
    light1_x, light1_y, light1_z, light1_i
])

material_box = widgets.VBox([
    widgets.HTML("<h3>Модель Блинн-Фонга</h3>"),
    ka_slider, kd_slider, ks_slider, shininess_slider
])

buttons_box = widgets.VBox([
    widgets.HTML("<h3>Действия</h3>"),
    render_button,
    save_button
])

# Создание макета в две колонки
left_column = widgets.VBox([screen_box, sphere_box, observer_box])
right_column = widgets.VBox([light_box, material_box, buttons_box])

main_layout = widgets.HBox([left_column, right_column])

display(main_layout)
display(output)

Output()

## Инструкции

1. Настройте параметры в панелях выше
2. Нажмите **"Рассчитать и визуализировать"** для генерации изображения
3. Нажмите **"Сохранить изображение"** для сохранения результатов

### Описание параметров:

- **Экран**: Размеры виртуального экрана и разрешение изображения
- **Сфера**: Координаты центра и радиус сферы
- **Наблюдатель**: Z-координата точки наблюдения (отрицательная)
- **Источник света**: Позиция и интенсивность единственного источника света
- **Материал**: 
  - Ka (ambient) = 0.0 для черного фона
  - Kd (diffuse) для диффузного рассеивания
  - Ks (specular) для зеркальных бликов
  - Shininess для фокусировки бликов

### Результаты визуализации:
- **Черно-белое изображение** - реалистичное распределение яркости
- **Цветовая карта** - для лучшего анализа градиентов
- **Срез яркости** - график распределения яркости вдоль центральной горизонтальной линии
- **Гистограмма** - статистическое распределение значений яркости